In [ ]:
pip install --upgrade coiled dask bokeh 

In [ ]:
!coiled login --token <>

In [64]:
import coiled 

cluster = coiled.Cluster(
    n_workers=50,
    configuration='lostmygithubaccount/isd',
)
cluster

Creating Cluster. This takes about a minute ...Checking environment images
Valid environment image found
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
ERROR:asyncio:_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
ERROR:asyncio:_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [65]:
from dask.distributed import Client

c = Client(cluster)
c

Client Scheduler: tls://ec2-18-191-149-183.us-east-2.compute.amazonaws.com:8786 Dashboard: https://cloud.coiled.io/dashboard/1038/status,Cluster Workers: 50 Cores: 50 Memory: 214.75 GB


In [26]:
!pip install --upgrade git+https://github.com/dask/adlfs fsspec

  Cloning https://github.com/dask/adlfs to /private/var/folders/gk/kpsykczx509cvpggyml79rsc0000gn/T/pip-req-build-3k__ht5q
Requirement already up-to-date: fsspec in /Users/cody/miniconda3/envs/dkdc/lib/python3.8/site-packages (0.8.3)
  Created wheel for adlfs: filename=adlfs-0.5.3+18.g65de4a6-py3-none-any.whl size=18432 sha256=7f41c5cf398a85294fde8821ce5e4360bbd0d85ba6e3e3cb33dd3a1db3a72ef0
  Stored in directory: /private/var/folders/gk/kpsykczx509cvpggyml79rsc0000gn/T/pip-ephem-wheel-cache-wzus60a9/wheels/a1/6c/2d/3b988aa0fd36594a0291addc195e10386681f4f354c95fbe75
Successfully built adlfs
  Attempting uninstall: adlfs
    Found existing installation: adlfs 0.5.3+18.g65de4a6
    Uninstalling adlfs-0.5.3+18.g65de4a6:
      Successfully uninstalled adlfs-0.5.3+18.g65de4a6


In [66]:
def update_packages():
    import os
    os.system('pip install --upgrade git+https://github.com/dask/adlfs fsspec')

In [68]:
import dask 

to_compute = [dask.delayed(update_packages) for i in range(50)]
c.compute(to_compute)

[<Future: pending, key: update_packages-8dc14def-6af4-4e8a-a86e-9772999e3a4d>,
 <Future: pending, key: update_packages-f7c63199-c86f-465f-b116-cf428f56b99b>,
 <Future: pending, key: update_packages-689855ad-296d-495e-a94e-ba3347b8f7aa>,
 <Future: pending, key: update_packages-03badb47-0073-45f7-9e3f-f919a07053aa>,
 <Future: pending, key: update_packages-39ac9038-6ab4-4036-ba01-f70f8568b688>,
 <Future: pending, key: update_packages-0e088806-d600-40f3-b842-429e5f02d667>,
 <Future: pending, key: update_packages-eb0a746c-13a3-4e86-bed1-b71ec2c01460>,
 <Future: pending, key: update_packages-1d576fff-3eb5-43c2-9e05-43123c1c226c>,
 <Future: pending, key: update_packages-975e7b9d-ae59-4f2c-85be-f91c657ce132>,
 <Future: pending, key: update_packages-cccd159e-fe93-4cff-a971-1cb57a437a35>,
 <Future: pending, key: update_packages-4f60c21f-e7de-4439-bdee-9621e34979d2>,
 <Future: pending, key: update_packages-84c87e00-dfb8-4d4f-acaf-90532f5d06ad>,
 <Future: pending, key: update_packages-5b8f6ab8-eb6

In [69]:
from adlfs import AzureBlobFileSystem

storage_options = {'account_name': 'azureopendatastorage'}

fs = AzureBlobFileSystem(**storage_options)
fs.ls('isdweatherdatacontainer')

['isdweatherdatacontainer/ISDWeather/',
 'isdweatherdatacontainer/ISDWeatherAll',
 'isdweatherdatacontainer/ISDWeatherAll/',
 'isdweatherdatacontainer/ISDWeatherBackup',
 'isdweatherdatacontainer/ISDWeatherBackup/',
 'isdweatherdatacontainer/ISDWeatherBackup_20200115/',
 'isdweatherdatacontainer/ISDWeatherTest/',
 'isdweatherdatacontainer/fama',
 'isdweatherdatacontainer/fama/']

In [35]:
import dask.dataframe as dd

df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv",
    dtype={
        "payment_type": "UInt8",
        "VendorID": "UInt8",
        "passenger_count": "UInt8",
        "RatecodeID": "UInt8",
    },
    storage_options={"anon": True},
    blocksize="16 MiB",
).persist()

df.groupby("passenger_count").tip_amount.mean().compute()

passenger_count
0    2.122789
1    2.206790
2    2.214306
3    2.137775
4    2.023804
5    2.235441
6    2.221105
7    6.675962
8    7.111625
9    7.377822
Name: tip_amount, dtype: float64

In [76]:
import dask.dataframe as dd 

df = dd.read_parquet('abfs://isdweatherdatacontainer/ISDWeather/year=2020/month=1/*.parquet', storage_options=storage_options, columns=['temperature'])
df

,temperature
npartitions=8,
,float64
,...
...,...
,...
,...


In [77]:
df.head()

KilledWorker: ("('read-parquet-head-1-5-read-parquet-43970d1e94ff6a7aee43dd9ff880414f', 0)", <Worker 'tls://10.2.12.37:46839', name: lostmygithubaccount-1038-worker-17-8581cf, memory: 0, processing: 1>)

In [61]:
df = df.persist()

In [62]:
c.restart()
c

Client Scheduler: tls://ec2-18-218-3-14.us-east-2.compute.amazonaws.com:8786 Dashboard: https://cloud.coiled.io/dashboard/1036/status,Cluster Workers: 25 Cores: 100 Memory: 429.50 GB
